Session 0.0: Load packages and customised functions

In [1]:
## a useful function to generate a data list for further analysis
import os,json,sys,itertools
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from locustvr_converter import preprocess_matrex_data
##need to add this additional cell because useful tools are in another folder. Need to integrate these two folders one day
current_working_directory = Path.cwd()
parent_dir = current_working_directory.resolve().parents[0]
sys.path.insert(0, str(parent_dir) + "\\utilities")
from useful_tools import select_animals_gpt,find_file

In [144]:
import matplotlib as mpl
from matplotlib import cm
class MplColorHelper:
    def __init__(self, cmap_name, start_val, stop_val):
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        return self.scalarMap.to_rgba(val)
colormap_name = "viridis"

Session 0.1: Load analysis methods in python dictionary form

In [2]:
json_file = "./analysis_methods_dictionary.json"
with open(json_file, "r") as f:
    analysis_methods = json.loads(f.read())
    

#Put the folder of your Unity experiment below
#thisDataset ="D:/MatrexVR_Swarm_Data/RunData"
#thisDataset ="D:/MatrexVR_blackbackground_Data/RunData"
thisDataset ="D:/MatrexVR_grass1_Data/RunData"
#thisDataset ="D:/MatrexVR_navigation_Data/RunData"
#parameter name means independent variable in the experiment
#parameter_name='kappa' 
parameter_name='mu'
#parameter_name='agent_speed'

Session 0.2: Load animals' experiment directory into a list

In [3]:
## this cell searches for a folder with csv files, usually that is the folder saving the tracking data.
## Since data from the 4 VRs are saved in the same folder, this command will return that one folder for the 4 experiment
dir_list = []
file_type=".csv"
for root, dirs, files in os.walk(thisDataset):
    for folder in dirs:
        folder_path=os.path.join(root,folder)
        if any(name.endswith(file_type) for name in os.listdir(folder_path)):
            dir_list.append(folder_path.replace("\\", "/"))


print(f"these directories are found {dir_list}")

these directories are found ['D:/MatrexVR_grass1_Data/RunData/20240907_142802', 'D:/MatrexVR_grass1_Data/RunData/20240907_170446', 'D:/MatrexVR_grass1_Data/RunData/20240907_190839', 'D:/MatrexVR_grass1_Data/RunData/20240908_125638', 'D:/MatrexVR_grass1_Data/RunData/20240908_150715', 'D:/MatrexVR_grass1_Data/RunData/20240908_174232', 'D:/MatrexVR_grass1_Data/RunData/20240908_193840']


In [4]:
##This cell is used to move data of the thermo-humidity logger to animals'folder
import shutil
#tmp_file_name='matrexVR240824-240901.txt'
tmp_file_name='DL220THP_Thermo2_241012_241014.csv'
tmp_source=os.path.join('Z:/Users/chiyu',tmp_file_name)
for this_dir in dir_list:
    tmp_new_dir = os.path.join(this_dir,tmp_file_name)
    if os.path.isfile(tmp_new_dir):
        print("Found EL USB temperature file in the new directory already")
    else:
        shutil.copy(tmp_source, tmp_new_dir)

Session 1.0: Create curated dataset based on a list of experiment directories

In [ ]:
#This function receives directory path that contains the 4-VR data and save the tracking + stimulus information as h5 file
pattern="VR*.h5"
for this_dir in dir_list:
    if "archive" in this_dir:
        print(f"skip archive folder for {this_dir}")
        continue
    if any(Path(this_dir).glob(pattern)) and analysis_methods.get("overwrite_curated_dataset")==False:
        print(f"curated matrexvr h5 database found in {this_dir}. Skip this file")
        continue
    else:
        print(f"no curated matrexvr h5 database in {this_dir}. Create curated file")
        preprocess_matrex_data(this_dir,analysis_methods)

Session 2.0: select animal based on condition and return which a directory list and a list of vr rig number to specify which animal to analyse

In [8]:
# build up dir_list of animals based on condition.
dir_list = []
file_type=".h5"
using_google_sheet=True
sheet_name = "Unity_MatrexVR"
scene_name=analysis_methods.get("experiment_name")
if analysis_methods.get("load_individual_data") == True:
    if using_google_sheet==True:
        database_id = "1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8"
                #https://docs.google.com/spreadsheets/d/1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8/edit?usp=sharing
        url = f"https://docs.google.com/spreadsheets/d/{database_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        #df = pd.read_excel(url, engine='openpyxl')## use this function if the file is not google sheet but uploaded excel file
        df = pd.read_csv(url)
    else:
        excel_file_path = "Z:/DATA/experiment_trackball_Optomotor/Locusts Management.xlsx"
        print(f"using a database {excel_file_path} from the server but this file might be outdated")
        # Create a 'with' statement to open and read the Excel file
        with pd.ExcelFile(excel_file_path) as xls:
            # Read the Excel sheet into a DataFrame with the sheet name (folder name)
            df = pd.read_excel(xls, sheet_name)
        ##list up the conditions and answers as strings for input argument to select animal. One condition must pair with one answer
    if analysis_methods.get("select_animals_by_condition") == True:
       #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)","gregarious_leader_grass","Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
       #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)","gregarious_leader_black")
        animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)",parameter_name,"Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
        #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)",parameter_name)
    else:
        animal_of_interest=df
    folder_name=animal_of_interest["folder name"].values
    dir_tile=np.tile(thisDataset, (len(folder_name), 1))
    vr_no=animal_of_interest["VR number"].values
    dir_list = [''.join([x[0], '/', y]) for x,y in zip(dir_tile,folder_name)]
    dir_dict = zip(dir_list, vr_no.tolist())
else:
    for root, dirs, files in os.walk(thisDataset):
        for folder in dirs:
            folder_path=os.path.join(root,folder)
            if any(name.endswith(file_type) for name in os.listdir(folder_path)):
                dir_list.append(folder_path.replace("\\", "/"))

Session 2.2: connecting information between two H5 files

Session 2.2.1: loading meta info from the json file (work in progress)

In [ ]:
#work in progress, this read condition directly from Control scene json file, which will be useful when not saving trial information into the simulated-agent file
'''
def read_condition_json(dir_list):
    j_pattern = f"*sequenceConfig.json"
    conditions_list=[]
    for this_dir in dir_list:
        found_result = find_file(Path(this_dir), j_pattern) 
        with open(found_result) as f:

            d = json.load(f)
            print(d["sequence"])

        for i in d["sequence"]:
            print("Mu:", i['Mu'])
            print("kappa:", i['kappa'])
            print("LocustSpeed:", i['LocustSpeed'])
            density = int(n_locusts.split(":")[1]) / (
            int(boundary_size.split(":")[1]) ** 2 / 10000
        )
            conditions = {
            "Density": density,
            mu.split(":")[0]: int(mu.split(":")[1]),
            "kappa": d["sequences"][0]['parameters']['kappa'],
            agent_speed.split(":")[0]: d["sequences"][0]['parameters']['LocustSpeed'],
        }
        conditions_list.append(conditions)
    return conditions_list
dir_list[0]
j_pattern = f"*sequenceConfig.json"
found_result = find_file(Path(this_dir), j_pattern)
test=pd.read_json(found_result)
print(test)
'''

Session 2.2.2: combine tables in the two lists in to 2 big tables

In [9]:
# When using spatial discretization, information about tracking quality is not logged in the dfXY, 
# hence there is a need to pass that information from df
def connect_two_tables(dir_list,analysis_methods,test_parameter='kappa',vr_no=[]):
    scene_name=analysis_methods.get("experiment_name")
    df_all=[]
    dfxy_all=[]
    dir_iterator=[]
    if len(vr_no)>0:
        print("i am using list")
        dir_iterator=zip(dir_list,vr_no)
    elif type(dir_list)==dict:
        print("i am using dictionary")
        dir_iterator=dir_dict
    else:
        print("there is a bug")
        return df_all,dfxy_all
    for this_dir,this_vr in dir_iterator:
        if Path(this_dir).is_dir()==False:
            print(f'no such a dir exist {this_dir}')
            continue
        summary_pattern = f"VR{this_vr}*score.h5"
        xy_pattern = f"VR{this_vr}*XY.h5"
        found_result = find_file(Path(this_dir), summary_pattern)        
        df = pd.read_hdf(found_result)
        df['VR'] = np.tile(f"VR{this_vr}", (len(df), 1))
        df['VR'] =df["VR"]+"_"+df["fname"]
        #COL = MplColorHelper(colormap_name, 0, num_independent_variable)
        found_result = find_file(Path(this_dir), xy_pattern)
        dfxy = pd.read_hdf(found_result)
        dfxy['VR'] = np.tile(f"VR{this_vr}", (len(dfxy), 1))
        dfxy['VR'] =dfxy["VR"]+"_"+dfxy["fname"]
        #df.loc[(df["distTotal"]<10.0) | (df["loss"]> 0.05), "distTotal"] = np.nan
        ##hardcode color code here for scatter plot
        if test_parameter == 'kappa':
            color_code={0.1: 0.2, 1.0: 0.4, 10.0: 0.6,100000.0:1}
        elif test_parameter == 'mu':
            if scene_name.lower()=='choice':
                color_code={0: 0.1,45: 0.4,315: 0.7}
            elif scene_name.lower()=='swarm':
                color_code={0: 0.1, 45: 0.2, 90: 0.3,135:0.4,180: 0.5, 225: 0.6, 270: 0.7,315:0.8}
            elif scene_name.lower()=='band':
                color_code={0: 0.1, 45: 0.2, 90: 0.3, 270: 0.7,315:0.8}
            else:
                return Warning('scene name not found')
        elif test_parameter == 'agent_speed':
            color_code={1.0: 0.2,2.0: 0.4, 4.0: 0.6,8.0:1}
        else:
            return Warning('test parameter not found')
        df['color_code'] = df[test_parameter].map(color_code)
        df_all.append(df)
        dfxy_all.append(dfxy)
    return df_all,dfxy_all

In [ ]:
if 'vr_no' in locals():
    df_all,dfxy_all=connect_two_tables(dir_list,analysis_methods,parameter_name,vr_no)
else:
    df_all,dfxy_all=connect_two_tables(dir_dict,analysis_methods,parameter_name)

Session 2.3: make summary plots of animals' response with customised plotting functions.

Session 2.3.1: introduce customised plotting functions used in Sercan's paper

In [87]:
#introduce customised functions
def plot_sercansincos(df,analysis_methods,parameters,parameter_name,vr_num='all'):
    save_output= analysis_methods.get("save_output")
    scene_name=analysis_methods.get("experiment_name")
    if analysis_methods.get("active_trials_only"):
        active_trial='active_trials'
    else:
        active_trial=''
    
    cos = df["cos"]
    sin = df["sin"]
    if 'density' in df.columns:
        density=df["density"].unique()[0]
        cos_fig_name=f"{vr_num}_cos_{scene_name}_{parameter_name}_{parameters}_density_{int(density)}{active_trial}.svg"
        sin_fig_name=f"{vr_num}_sin_{scene_name}_{parameter_name}_{parameters}_density_{int(density)}{active_trial}.svg" 
    elif scene_name=='choice':
        cos_fig_name=f"{vr_num}_cos_{scene_name}_{parameter_name}_{parameters}_single_target_{df['type'].values[0]}{active_trial}.svg"
        sin_fig_name=f"{vr_num}_sin_{scene_name}_{parameter_name}_{parameters}_single_target_{df['type'].values[0]}{active_trial}.svg"

    fig, ax = plt.subplots(dpi=300, figsize=(1.1,0.25))
    #plt.rcParams.update(plt.rcParamsDefault)
    plt.rcParams.update({'font.size': 8})
    plt.rcParams['font.family'] = 'Arial'
    plt.set_cmap('cividis')
    # Set the axis line width to 2
    plt.rcParams['ytick.major.width'] = 2
    plt.rcParams['xtick.major.width'] = 2
    plt.rcParams['axes.linewidth'] = 2
    plt.subplots_adjust(bottom=0.4)
    sns.kdeplot(cos, cut=0, color="#21918c", fill=True, alpha=0.9)
    plt.xlim(-1,1)
    plt.title("r cos\u03F4")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_yticks([])
    plt.ylabel("")
    plt.xlabel("")    
    if save_output==True:
        plt.savefig(cos_fig_name)
    fig, ax = plt.subplots(dpi=300, figsize=(1.1,0.25))
    plt.subplots_adjust(bottom=0.4)
    sns.kdeplot(sin, cut=0, color="#21918c",  fill=True, alpha=0.9)#),lw=1,
    plt.xlim(1,-1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    plt.xticks(rotation = 90)
    ax.set_yticks([])
    plt.ylabel("")
    plt.xlabel("")
    plt.title("r sin\u03F4")
    if save_output==True:
        plt.savefig(sin_fig_name)
    plt.show()
def plot_sercantrajec(dfXY,analysis_methods,parameters,parameter_name,trajec_lim=1000,vr_num='all'):
    save_output= analysis_methods.get("save_output")
    scene_name=analysis_methods.get("experiment_name")
    if analysis_methods.get("active_trials_only"):
        active_trial='active_trials'
    else:
        active_trial=''
    
    a = dfXY.groupby('VR')
    if 'density' in dfXY.columns:
        density=dfXY["density"].unique()[0]
        print(density)
        fig_name=f"{vr_num}_summary_trajectory_{scene_name}_{parameter_name}_{parameters}_density_{int(density)}{active_trial}.png"
    else:
        fig_name=f"{vr_num}_summary_trajectory_{scene_name}_{parameter_name}_{parameters}_single_target_{dfXY['type'].values[0]}{active_trial}.png"

    fig, ax = plt.subplots(figsize=(3,3), dpi=300) 
    plt.rcParams.update(plt.rcParamsDefault)
    plt.rcParams.update({'font.size': 8})
    plt.rcParams['ytick.major.width'] = 2
    plt.rcParams['xtick.major.width'] = 2
    plt.rcParams['axes.linewidth'] = 2
    # Get the colormap
    cmap = plt.get_cmap('viridis')

    #plt.style.use('dark_background') 
    for i, (key2, grp2) in enumerate(a):
        color = cmap(i/ len(a))
        plt.plot(grp2["X"].values, grp2["Y"].values, color=color, linewidth=1)


        # Here plot agent's trajectory with hardcode parameters
    if scene_name=='choice' and dfXY['type'].values[0] !='empty_trial':
        agent_speed=2
        radial_distance=8
        duration=60
        travel_direction=parameters*-np.pi/180#the radian circle is clockwise in Unity, so 45 degree should be used as -45 degree in the regular radian circle
        radial_distance_b=np.cos(travel_direction)*radial_distance
        delta_cos=np.cumsum(np.repeat(np.cos(travel_direction)*agent_speed, duration))
        agent_cos=radial_distance_b+delta_cos
        radial_distance_b=np.sin(travel_direction)*radial_distance
        delta_sin=np.cumsum(np.repeat(np.sin(travel_direction)*agent_speed, duration))
        agent_sin=radial_distance_b+delta_sin
        plt.plot(agent_cos, agent_sin, color='k', linewidth=1)

    plt.xlim(-1*trajec_lim, trajec_lim)
    plt.ylim(-1*trajec_lim, trajec_lim)
    plt.yticks([-1*trajec_lim, 0, trajec_lim])
    plt.xticks([-1*trajec_lim, 0, trajec_lim])
        # Set the aspect ratio to be equal
    plt.gca().set_aspect('equal')                                                  
    if save_output==True:    
        plt.savefig(fig_name)
    plt.show()

Session 2.3.2: introduce a pilot scatter plot functions

In [147]:
def plot_travel_distance_set(df_all,analysis_methods,parameter_name,y_axis_lim=[0.1,1000]):

    COL = MplColorHelper(colormap_name, 0, 11)
    
    fig, (ax1, ax2,ax3) = plt.subplots(
    nrows=1, ncols=3, figsize=(18, 6), tight_layout=True
)
    colour_code=analysis_methods.get("graph_colour_code")
    if parameter_name=='kappa':
        ax1.set_xscale('log')
    #ax1.set_yscale('log')
    ax1.set_ylim([y_axis_lim[0],y_axis_lim[1]])
    ax1.set(
        yticks=[y_axis_lim[0], y_axis_lim[1]],
        ylabel="Travel distance per second",
        xticks=sorted(df_all[0][parameter_name].unique()),
        xlabel=parameter_name,
    )
    ax2.set(
        ylabel="trial n",
        xlabel="interval prior to trial n",
        xlim=[y_axis_lim[0],y_axis_lim[1]],
        ylim=[y_axis_lim[0],y_axis_lim[1]],
        title="travel distance per second",
    )
    ax3.set(
        ylabel="trial n",
        xlabel="trial n-1",
        xlim=[y_axis_lim[0],y_axis_lim[1]],
        ylim=[y_axis_lim[0],y_axis_lim[1]],
        title="travel distance per second",
    )
    #ax1.gca().set_aspect('equal')
    for id in np.arange(len(df_all)):
        df=df_all[id]
        viridis_code=df['color_code']*10#normalise the code from 1 to 10
        viridis_code=viridis_code.astype('int')
        #set some thresholds to remove bad tracking 
        df.loc[(df["distTotal"]<50.0) | (df["loss"]> 0.05), "distTotal"] = np.nan
        if df.iloc[0]["VR"].startswith('VR1'):
            vr_color=colour_code[0]
        elif df.iloc[0]["VR"].startswith('VR2'):
            vr_color=colour_code[1]
        elif df.iloc[0]["VR"].startswith('VR3'):
            vr_color=colour_code[2]
        else:
            vr_color=colour_code[3]

        #ax1.scatter(df.iloc[::2][parameter_name], df.iloc[::2]["distTotal"]/df.iloc[::2]["duration"],c='k')

        ax1.scatter(df.iloc[1::2][parameter_name], df.iloc[1::2]["distTotal"]/df.iloc[1::2]["duration"],c=COL.get_rgb(viridis_code[1::2]))
        if df.shape[0] % 2 == 0:
            ax2.scatter(df.iloc[::2]["distTotal"]/df.iloc[::2]["duration"],df.iloc[1::2]["distTotal"]/df.iloc[1::2]["duration"],c=COL.get_rgb(viridis_code[1::2]))
        else:
            ax2.scatter(df.iloc[:-1:2]["distTotal"]/df.iloc[:-1:2]["duration"],df.iloc[1::2]["distTotal"]/df.iloc[1::2]["duration"],c=COL.get_rgb(viridis_code[1::2]))
        ax3.scatter(df.iloc[1:-2:2]["distTotal"]/df.iloc[1:-2:2]["duration"],df.iloc[3::2]["distTotal"]/df.iloc[3::2]["duration"],c=COL.get_rgb(viridis_code[3::2]))
    plt.show()

In [ ]:
#plot responses (mean angle and travel distance) from individual experiments (usually every 4 animal an experiment; different colour mark different animals in that experiment) 
#or comparing trial by trial response through normalised response (e.g. ratio to previous trial) or scatter plot (each dot means a comparison, 
#different colour means data from different rigs, different independent variables is marked with different kappa value)
## 1st: plots with independent variables such as kappa or mu against travel distance or angle
if len(df_all)>0:
    plot_travel_distance_set(df_all,analysis_methods,parameter_name,y_axis_lim=[0,12])

Session 2.3.3: combine pandas dataframe across animals and filter out trials with bad tracking (these are preprocessing steps to use Sercan's functions)

In [111]:
#Firstly, concatenate every animal's dataframe into a big table and then sort them based on conditions.
if len(dfxy_all)>0:
    dfxy_con = pd.concat(dfxy_all)
if len(df_all)>0:
    df_con = pd.concat(df_all)
good_tracking=df_con['loss']< 0.05
active_trials=(df_con['loss'] < 0.05) & (df_con["distTotal"]>50)

In [132]:
analysis_methods.update({"save_output": False})

Session 2.3.4: plot trial by trial trajectory

In [ ]:
#plotting trajectory
#differentiate between stim and ISI based on columns in dfxy_con
if analysis_methods.get("active_trials_only"):
    df_interest=df_con[active_trials]
else:
    df_interest=df_con[good_tracking]


if analysis_methods.get("experiment_name")=="choice":
    stim_or_isi=dfxy_con['radial_distance']
elif analysis_methods.get("experiment_name")=="swarm" or analysis_methods.get("experiment_name")=="band":
    stim_or_isi=dfxy_con['density']
df_stim=dfxy_con.loc[(dfxy_con['VR'].isin(df_interest['VR'])) & (stim_or_isi>0)]
for key, grp in df_stim.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_sercantrajec(grp,analysis_methods,key,parameter_name,500)
df_isi=dfxy_con.loc[(dfxy_con['VR'].isin(df_interest["VR"])) & (stim_or_isi==0)]
for key, grp in df_isi.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_sercantrajec(grp,analysis_methods,key,parameter_name,500)
#xy_lim at around 2000 is good for trial lasts around 4 or 5 min
#xy_lim at around 500 is good for trial lasts around 1 min

Session 2.3.5: pool mean angle together to make KDE plots

In [ ]:
#Visualise the distribution of mean angle at sin and cos using seaborn kernel density estimation plot
#differentiate between stim and ISI based on columns in df_con
if analysis_methods.get("active_trials_only"):
    df_interest=df_con[active_trials]
else:
    df_interest=df_con

if analysis_methods.get("experiment_name")=="choice":
    stim_or_isi=df_interest['radial_distance']
elif analysis_methods.get("experiment_name")=="swarm" or analysis_methods.get("experiment_name")=="band":
    stim_or_isi=df_interest['density']
df_stim=df_interest[stim_or_isi>0]
for key, grp in df_stim.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_sercansincos(grp,analysis_methods,key,parameter_name)
df_isi=df_interest[stim_or_isi==0]
for key, grp in df_isi.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_sercansincos(grp,analysis_methods,key,parameter_name)

In [125]:
def plot_circular_histrogram(df,analysis_methods,parameters,parameter_name,vr_num='all'):
    save_output= analysis_methods.get("save_output")
    scene_name=analysis_methods.get("experiment_name")
    if analysis_methods.get("active_trials_only"):
        active_trial='active_trials'
    else:
        active_trial=''
    
    angles = df["mean_angle"]
    if 'density' in df.columns:
        density=df["density"].unique()[0]
        hist_fig_name=f"{vr_num}_circular_hist_{scene_name}_{parameter_name}_{parameters}_density_{int(density)}{active_trial}.svg"
    elif scene_name=='choice':
        hist_fig_name=f"{vr_num}_circular_hist_{scene_name}_{parameter_name}_{parameters}_single_target_{df['type'].values[0]}{active_trial}.svg"

    ax = plt.subplot(111, polar=True)
    ax.hist(angles, bins=24, alpha=0.75)
    #ax.set_xticks([])
    ax.set_yticks([5,10])
    ax.set_xticklabels([])
    #ax.set_title(f'banding direction: {parameters}')
    if save_output==True:
        plt.savefig(hist_fig_name)
    plt.show()
def plot_travel_histrogram(df,analysis_methods,parameters,parameter_name,vr_num='all'):
    save_output= analysis_methods.get("save_output")
    scene_name=analysis_methods.get("experiment_name")
    if analysis_methods.get("active_trials_only"):
        active_trial='active_trials'
    else:
        active_trial=''
    distTotal = df["distTotal"]
    if 'density' in df.columns:
        density=df["density"].unique()[0]
        hist_fig_name=f"{vr_num}_travel_hist_{scene_name}_{parameter_name}_{parameters}_density_{int(density)}{active_trial}.svg"
    elif scene_name=='choice':
        hist_fig_name=f"{vr_num}_travel_hist_{scene_name}_{parameter_name}_{parameters}_single_target_{df['type'].values[0]}{active_trial}.svg"

    #fig, ax = plt.subplots(dpi=300, figsize=(1,1))
    fig, ax = plt.subplots(figsize=(5,2))
    ax.hist(distTotal, bins=24, alpha=0.75)
    ax.set_xticks([50,100])
    #ax.set_yticks([5,10])
    #ax.set_xticklabels([])
    #ax.set_title(f'banding direction: {parameters}')
    if save_output==True:
        plt.savefig(hist_fig_name)
    plt.show()

In [ ]:
#Visualise the distribution of  mean angle using seaborn kernel density estimation plot
#differentiate between stim and ISI based on columns in df_con
if analysis_methods.get("active_trials_only"):
    df_interest=df_con[active_trials]
else:
    df_interest=df_con
if analysis_methods.get("experiment_name")=="choice":
    stim_or_isi=df_interest['radial_distance']
elif analysis_methods.get("experiment_name")=="swarm" or analysis_methods.get("experiment_name")=="band":
    stim_or_isi=df_interest['density']
#df_stim=df_con[stim_or_isi>0]
df_stim=df_interest[stim_or_isi>0]
for key, grp in df_stim.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_travel_histrogram(grp,analysis_methods,key,parameter_name)
    #plot_circular_histrogram(grp,analysis_methods,key,parameter_name)
#df_isi=df_con[stim_or_isi==0]
df_isi=df_interest[stim_or_isi==0]
for key, grp in df_isi.groupby(parameter_name):
    print(f"{parameter_name}:{key}")
    plot_travel_histrogram(grp,analysis_methods,key,parameter_name)
    #plot_circular_histrogram(grp,analysis_methods,key,parameter_name)

Session 2.3.5: plot individual animal's trajectory

In [ ]:
dir_iterator=[]
if len(vr_no)>0:
    print("i am using list")
    dir_iterator=zip(dir_list,vr_no)
elif type(dir_list)==dict:
    print("i am using dictionary")
    dir_iterator=dir_dict
else:
    print("there is a bug")

for this_dir,this_vr in dir_iterator:
    if Path(this_dir).is_dir()==False:
        print(f'no such a dir exist {this_dir}')
        continue
    locust_pattern = f"VR{this_vr}*XY.h5"
    found_result = find_file(Path(this_dir), locust_pattern)        
    print(found_result)
    dfxy = pd.read_hdf(found_result)
    dfxy['VR'] = np.tile(f"VR{this_vr}", (len(dfxy), 1))
    dfxy['VR'] =dfxy["VR"]+"_"+dfxy["fname"]
    summary_pattern = f"VR{this_vr}*score.h5"
    found_result = find_file(Path(this_dir), summary_pattern)        
    df = pd.read_hdf(found_result)

    df['VR'] = np.tile(f"VR{this_vr}", (len(df), 1))
    df['VR'] =df["VR"]+"_"+df["fname"]

    if analysis_methods.get("experiment_name")=="choice":
        stim_or_isi=dfxy['radial_distance']
    elif analysis_methods.get("experiment_name")=="swarm" or analysis_methods.get("experiment_name")=="band":
        stim_or_isi=dfxy['density']
    df_stim=dfxy.loc[(dfxy['VR'].isin(df['VR'])) & (stim_or_isi>0)]
    for key, grp in df_stim.groupby(parameter_name):
        print(f"kappa:{key}")
        plot_sercantrajec(grp,analysis_methods,key,parameter_name,500)
    df_isi=dfxy.loc[(dfxy['VR'].isin(df['VR'])) & (stim_or_isi==0)]
    for key, grp in df_isi.groupby(parameter_name):
        print(f"kappa:{key}")
        plot_sercantrajec(grp,analysis_methods,key,parameter_name,500)

Session 2.3.6: plot individual animal's kernel density estimation plot

In [ ]:
dir_iterator=[]
if len(vr_no)>0:
    print("i am using list")
    dir_iterator=zip(dir_list,vr_no)
elif type(dir_list)==dict:
    print("i am using dictionary")
    dir_iterator=dir_dict
else:
    print("there is a bug")

for this_dir,this_vr in dir_iterator:
    if Path(this_dir).is_dir()==False:
        print(f'no such a dir exist {this_dir}')
        continue
    locust_pattern = f"VR{this_vr}*score.h5"
    found_result = find_file(Path(this_dir), locust_pattern)        
    df = pd.read_hdf(found_result)
    if analysis_methods.get("experiment_name")=="choice":
        stim_or_isi=df['radial_distance']
    elif analysis_methods.get("experiment_name")=="swarm" or analysis_methods.get("experiment_name")=="band":
        stim_or_isi=df['density']
    df_stim=df[stim_or_isi>0]
    for key, grp in df_stim.groupby(parameter_name):
        print(f"{parameter_name}:{key}")
        plot_sercansincos(grp,analysis_methods,key,parameter_name)
    df_isi=df[stim_or_isi==0]
    for key, grp in df_isi.groupby(parameter_name):
        print(f"{parameter_name}:{key}")
        plot_sercansincos(grp,analysis_methods,key,parameter_name)

[Optional] Analyse data with multi-engines

In [ ]:
##this cell start the multi-engines. Make sure to run only once
import time
import ipyparallel as ipp
def show_clusters():
    clusters = ipp.ClusterManager().load_clusters() 
    print("{:15} {:^10} {}".format("cluster_id", "state", "cluster_file")) 
    for c in clusters:
        cd = clusters[c].to_dict()
        cluster_id = cd['cluster']['cluster_id']
        controller_state = cd['controller']['state']['state']
        cluster_file = getattr(clusters[c], '_trait_values')['cluster_file']
        print("{:15} {:^10} {}".format(cluster_id, controller_state, cluster_file))
    return cluster_id

cluster = ipp.Cluster(n=6)
await cluster.start_cluster()
cluster_neuropc=show_clusters()

In [ ]:
##input cluster_id from previous cell
rc = ipp.Client(cluster_id=cluster_neuropc)

# Create a DirectView for parallel execution
dview = rc.direct_view()

# Define a function for parallel processing
def process_directory(this_dir, analysis_methods):
    from pathlib import Path
    import sys
    current_working_directory = Path.cwd()
    parent_dir = current_working_directory.resolve().parents[0]
    sys.path.insert(0, str(parent_dir) + "\\utilities")
    from locustvr_converter import preprocess_matrex_data
    preprocess_matrex_data(this_dir,analysis_methods)

# Define analysis_methods

# Use parallel execution to process directories
dview.map_sync(process_directory, dir_list, [analysis_methods] * len(dir_list))

In [ ]:
rc.shutdown()